In [84]:
import requests
from bs4 import BeautifulSoup
from io import StringIO
import pandas as pd

In [66]:
url = "https://www.wis-tns.org/astronotes?&date_start%5Bdate%5D=2024-07-01" #&date_end%5Bdate%5D=2024-10-04&keywords%5Bvalue_field%5D=%22%22%22%22%20%22%22TDE%22%22&num_page=50"

In [67]:
hdr = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
res = requests.get(url, headers=hdr)
res

<Response [200]>

In [68]:
soup = BeautifulSoup(res.content.decode())

In [69]:
tde_res = soup.find(string="TDE").find_parents()

In [112]:
astronotes = soup.find_all(class_="note")
t = {"TDE", "TDE-H", "TDE-He"}
for a in astronotes:
    r = a.find(string="TDE")
    if r is None: continue

    partial_link = a.find(class_="note-link").get("href")        
    link = f"https://www.wis-tns.org{partial_link}"
    print(link)

    authors = a.find(class_="note-coauthors").get_text()
    print(authors)

    title = a.find(class_="note-title").get_text()
    print(title)

    note = requests.get(link, headers=hdr).content.decode()
    subsoup = BeautifulSoup(note)

    tab = subsoup.find(class_="objects-table")
    df = pd.read_html(StringIO(str(tab)))[0]
    
    # only take TDEs
    # if "Candidate Type"
    class_names = []
    if "Reported Obj-Type" in df:
        class_names.append("Reported Obj-Type")
    if "TNS Obj-Type" in df:
        class_names.append("TNS Obj-Type")
    if "Candidate type" in df:
        class_names.append("Candidate type")
        
    filt = df[class_names].isin(t).sum(axis=1).astype(bool)
    df = df[filt]

    for _, row in df.iterrows():
        row = row.dropna()
        #print(row)
        name = row.Name

        ra, dec = None, None
        if "TNS RA" in row and "TNS DEC" in row:
            ra = row["TNS RA"]
            dec = row["TNS DEC"]
        elif "Reported RA" in row and "Reported Dec" in row:
            ra = row["Reported RA"]
            dec = row["Reported DEC"]

        classification = None
        for pkey in ["Reported Obj-Type", "TNS Obj-Type", "Candidate type"]:
            if pkey in row:
                classification = row[pkey]

        if classification not in t: continue
        
        redshift = None
        for col in row.keys():
            if "Redshift" in col:
                redshift = row[col]
        
        print(name)
        print(ra, dec)
        print(classification)
        print(redshift)
        print()

    print()
    print('####################################################################')
    print()

https://www.wis-tns.org/astronotes/astronote/2024-282
X. Sheng, A. Aamer, M. Nicholl, K. W. Smith, C. R. Angus, M. Fulton, T. Moore, J. Weston, D. R. Young (QUB), P. Ramsd...
SLSN and TDE candidates from NEEDLE on Lasair
2024vya [ZTF24abimtdl]
02:37:26.434 -16:34:48.66
TDE
None

2024vpg [ZTF24abhtadx]
19:47:23.194 +57:35:28.84
TDE
None


####################################################################

https://www.wis-tns.org/astronotes/astronote/2024-268
A. Sankar. K (NCU), X. Sheng, M. Nicholl, K. W. Smith, C. R. Angus, A. Aamer, M. Fulton, T. Moore, J. Weston, D. R. Y...
SLSN and TDE candidates from NEEDLE on Lasair
2024vsb [ZTF24abhfqms]
21:46:03.598 -08:43:01.43
TDE
None

2024vwo [ZTF24abikscm]
22:04:50.233 -02:49:40.49
TDE
None


####################################################################

https://www.wis-tns.org/astronotes/astronote/2024-263
Noah Franz (Arizona), Kate D. Alexander (Arizona), Collin T. Christy (Arizona), Suvi Gezari (STScI/JHU), Adelle J. Go...
Radio

In [114]:
import time
from astropy.time import Time
Time(time.time())

ValueError: Input values did not match any of the formats where the format keyword is optional:
- 'datetime': Input values for datetime class must be datetime objects
- 'ymdhms': input must be dict or table-like
- 'iso': Input values for iso class must be strings
- 'isot': Input values for isot class must be strings
- 'yday': Input values for yday class must be strings
- 'datetime64': Input values for datetime64 class must be datetime64 objects
- 'fits': Input values for fits class must be strings
- 'byear_str': Input values for byear_str class must be strings
- 'jyear_str': Input values for jyear_str class must be strings
- 'astropy_time': Input values for astropy_time class must all be the same astropy Time type.

In [122]:
from datetime import datetime, timedelta

In [125]:
now = datetime.now()
dt = timedelta(days=2)
qdate = now-dt
qdate_strfmt = f"{qdate.year}-{qdate.month}-{qdate.day}"
strfmt

'2024-10-2'